# Basic Setup

Import code from either Google Colab or local drive.
Select that option by either executing the first or second cell.

In [0]:
# SET HERE if notebook gets executed on google colab or locally
is_on_colab = True

In [86]:
if is_on_colab:
    # Google Colab setup
    from google.colab import drive
    drive.mount('/content/drive')

    import os
    os.chdir("/content/drive/My Drive/adl4cv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# ONLY NECESSARY FOR LOCAL EXECUTION (WORKS WITHOUT THIS CELL IN GOOGLE COLAB)
# Setup that is necessary for jupyter notebook to find sibling-directories
# see: https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
if not is_on_colab:
    
    import os
    import sys
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

In [0]:
# Imports for this notebook

from networks.baseline import BaselineModel
from training.solver import Solver
from training.single_image_dataloader import FaceForensicsImagesDataset, ToTensor
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler
import torch

In [89]:
# Check training on GPU?

cuda = torch.cuda.is_available()

print("Training is on GPU with CUDA: {}".format(cuda))

device = "cuda:0" if cuda else "cpu"

print("Device: {}".format(device))

Training is on GPU with CUDA: True
Device: cuda:0


# Load Data and Model

Load FaceForensics sequences.
Choose a list of corresponding file-paths.

Load the model for this notebook.

In [90]:
# Load Dataset from drive location

# Decide here if using the 100 videos dataset or the 1000 videos dataset
is_large_dataset = False # because we constructed two different folders, one with 100 videos, one with 1000 videos

root_path = "/content/drive/My Drive/" if is_on_colab else "F:/Google Drive/"
root_dir = root_path # saved for stuff that does not need the faceforensics suffix in it, e.g. in model saving

dataset_root = "FaceForensics_large" if is_large_dataset else "FaceForensics_Sequences"
root_path += dataset_root

sequence = "sequences_299x299_10seq@10frames_skip_5_uniform" if is_large_dataset else "sequences_299x299_5seq@10frames_skip_5_uniform"
#sequence = "sequences_299x299_10seq@5frames_skip_1_uniform"

original_location = "/original_sequences/youtube/c40/" + sequence
deepfake_location = "/manipulated_sequences/Deepfakes/c40/" + sequence
face2face_location = "/manipulated_sequences/Face2Face/c40/" + sequence
faceswap_location = "/manipulated_sequences/FaceSwap/c40/" + sequence
neuraltextures_location = "/manipulated_sequences/NeuralTextures/c40/" + sequence

locations = [original_location, deepfake_location] #deepfake_location, face2face_location, faceswap_location

train_loc = [root_path + s + ("/train" if is_large_dataset else "") for s in locations]
val_loc = [root_path + s + ("/val" if is_large_dataset else "") for s in locations] 

train_dict = {"train-dataset-" + str(i): train_loc[i] for i in range(len(train_loc))}
val_dict = {"val-dataset-" + str(i): val_loc[i] for i in range(len(val_loc))}

# when using two fake variants: multiply fake-loss by 0.5 to account for twice as many fake than original samples
fake_weight_factor = 1.0 / (len(train_loc) - 1)

data_dict = {"fake_weight_factor": fake_weight_factor, **train_dict, **val_dict}


train_dataset = FaceForensicsImagesDataset(train_loc,
                                     transform=ToTensor())

val_dataset = FaceForensicsImagesDataset(val_loc,
                                     transform=ToTensor())

print("Loaded following data: {}".format(data_dict))

Loaded following data: {'fake_weight_factor': 1.0, 'train-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'train-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform'}


In [91]:
# from: https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

dataset_args = {
    "batch_size": 32,
    "validation_percentage": 0.2,
    "sequence": sequence,
    **data_dict
}

# Should set num_workers=0, otherwise the caching in the dataset does not work... but why?
num_workers = 4

if is_large_dataset:
    # here we have separate folders in the dataset: /train and /val --> use all of it for both datasets

    # Should set num_workers=0, otherwise the caching in the dataset does not work... but why?
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=dataset_args["batch_size"], 
                                               num_workers=num_workers,
                                               shuffle=True)

    validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=dataset_args["batch_size"],
                                                    num_workers=num_workers,
                                                    shuffle=True)
else:
    # here we have one folder for train+val and must split it accordingly ourselfes at runtime
    # Creating data indices for training and validation splits:
    train_indices, val_indices = train_dataset.get_train_val_lists(1 - dataset_args["validation_percentage"], dataset_args["validation_percentage"])

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Should set num_workers=0, otherwise the caching in the dataset does not work... but why?
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=dataset_args["batch_size"], 
                                               num_workers=num_workers,
                                               sampler=train_sampler)

    validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=dataset_args["batch_size"],
                                                    num_workers=num_workers,
                                                    sampler=val_sampler)

dataset_args["train_len"] = len(train_loader)
dataset_args["val_len"] = len(validation_loader)

print("Dataset parameters: {}".format(dataset_args))

Dataset parameters: {'batch_size': 32, 'validation_percentage': 0.2, 'sequence': 'sequences_299x299_5seq@10frames_skip_5_uniform', 'fake_weight_factor': 1.0, 'train-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'train-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'train_len': 225, 'val_len': 57}


In [92]:
# Load baseline model
model_args={
    "model_choice": "xception",
    "num_out_classes": 2,
    "dropout": 0.5
}

model = BaselineModel(model_choice=model_args["model_choice"],
                      num_out_classes=model_args["num_out_classes"],
                      dropout=model_args["dropout"])
model.train_only_last_layer()

model_args["model"] = type(model).__name__

print("Model configuration: {}".format(model_args))

print("Only the following layers require gradient backpropagation (param.requires_grad)")
for name, param in model.model.named_parameters():
    if param.requires_grad:
        print("param: {} requires_grad: {}".format(name, param.requires_grad))

Using dropout 0.5
Model configuration: {'model_choice': 'xception', 'num_out_classes': 2, 'dropout': 0.5, 'model': 'BaselineModel'}
Only the following layers require gradient backpropagation (param.requires_grad)
param: last_linear.1.weight requires_grad: True
param: last_linear.1.bias requires_grad: True


# Training Visualization

Start Tensorboard for visualization of the upcoming training / validation / test steps.

In [0]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir runs

# Training

Start training process.

In [93]:
# Create unique ID for this training process for saving to disk.

from datetime import datetime
import uuid
now = datetime.now() # current date and time
id = str(uuid.uuid1())
id_suffix = now.strftime("%Y-%b-%d_%H-%M-%S") + "_" + id

log_dir = "runs/Baseline/" + id_suffix
print("log_dir:", log_dir)

log_dir: runs/Baseline/2020-Jan-26_20-15-00_87e423d0-4078-11ea-b304-0242ac1c0002


In [94]:
# Configure solver
extra_args = {
    **model_args,
    **dataset_args
}

weights = [dataset_args["fake_weight_factor"], 1.0]
class_weights = torch.FloatTensor(weights).to(device)

print("Using following weighting scheme in cross-entropy-loss: {}\n".format(class_weights))

solver = Solver(optim=torch.optim.Adam,
                optim_args={ "lr": 2e-4,
                             "betas": (0.9, 0.999),
                             "eps": 1e-8,
                             "weight_decay": 0.1}, # is the l2 regularization parameter, see: https://pytorch.org/docs/stable/optim.html
                loss_func=torch.nn.CrossEntropyLoss(weight=class_weights),
                extra_args=extra_args,
                log_dir=log_dir)

Using following weighting scheme in cross-entropy-loss: tensor([1., 1.], device='cuda:0')

Hyperparameters of this solver: {'loss function': 'CrossEntropyLoss', 'optimizer': 'Adam', 'learning rate': 0.0002, 'weight_decay': 0.1, 'model_choice': 'xception', 'num_out_classes': 2, 'dropout': 0.5, 'model': 'BaselineModel', 'batch_size': 32, 'validation_percentage': 0.2, 'sequence': 'sequences_299x299_5seq@10frames_skip_5_uniform', 'fake_weight_factor': 1.0, 'train-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'train-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-0': '/content/drive/My Drive/FaceForensics_Sequences/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', 'val-dataset-1': '/content/drive/My Drive/FaceForensics_Sequences/manipulated_sequences/Deepfake

In [95]:
# Start training

solver.train(model, train_loader, validation_loader, num_epochs=15, log_nth=100)

START TRAIN on device: cuda:0


[Iteration 1/225] TRAIN loss: 0.6740680932998657
[Iteration 101/225] TRAIN loss: 0.6720247864723206
[Iteration 201/225] TRAIN loss: 0.5987541079521179

[EPOCH 1/15] TRAIN mean acc/loss: 0.6513396057347669/0.6396249532699585


[Iteration 1/57] Val loss: 0.5795182585716248

[EPOCH 1/15] VAL mean acc/loss: 0.6754385964912281/0.627863883972168


[Iteration 1/225] TRAIN loss: 0.6013601422309875
[Iteration 101/225] TRAIN loss: 0.5674222707748413
[Iteration 201/225] TRAIN loss: 0.5394569635391235

[EPOCH 2/15] TRAIN mean acc/loss: 0.7287141577060933/0.5782613158226013


[Iteration 1/57] Val loss: 0.5883520245552063

[EPOCH 2/15] VAL mean acc/loss: 0.7006578947368421/0.6048509478569031


...

[Iteration 1/57] Val loss: 0.5700605511665344

[EPOCH 13/15] VAL mean acc/loss: 0.6847587719298246/0.5931963324546814


[Iteration 1/225] TRAIN loss: 0.49753814935684204
[Iteration 101/225] TRAIN loss: 0.5791372060775757
[Iteration 201/225] TRAIN loss: 0.5184728503227234

[EPOCH 14/15] TRAIN mean acc/loss: 0.7762320788530466/0.5051231384277344


[Iteration 1/57] Val loss: 0.6900990605354309

[EPOCH 14/15] VAL mean acc/loss: 0.6896929824561403/0.593173623085022


[Iteration 1/225] TRAIN loss: 0.4592111110687256
[Iteration 101/225] TRAIN loss: 0.5304315090179443
[Iteration 201/225] TRAIN loss: 0.5234341025352478

[EPOCH 15/15] TRAIN mean acc/loss: 0.7692652329749103/0.5131861567497253


[Iteration 1/57] Val loss: 0.5868423581123352

[EPOCH 15/15] VAL mean acc/loss: 0.6935307017543859/0.5924040079116821
FINISH.


# Test

Test with same fake set and different fake set as for training.
Will load the data and start the training.

Visualizations can be seen in Tensorboard above.

In [96]:
# Load test data for SAME FAKE SET

if is_large_dataset:
    same_test_data_location = [root_path + s + "/test" for s in locations]
else:
    same_test_data_location = [root_path + "/FaceForensics_Testset" + s for s in locations]

same_test_dataset = FaceForensicsImagesDataset(same_test_data_location, transform=ToTensor())

same_test_loader = torch.utils.data.DataLoader(same_test_dataset, batch_size=dataset_args["batch_size"], 
                                               shuffle=True,
                                               num_workers=4)

print("Length of same fake test set: {}".format(len(same_test_dataset)))
print("Loaded test set: {}".format(same_test_data_location))

Length of same fake test set: 1000
Loaded test set: ['/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/original_sequences/youtube/c40/sequences_299x299_5seq@10frames_skip_5_uniform', '/content/drive/My Drive/FaceForensics_Sequences/FaceForensics_Testset/manipulated_sequences/Deepfakes/c40/sequences_299x299_5seq@10frames_skip_5_uniform']


In [97]:
# Start testing

solver.test(model, same_test_loader, test_prefix="Same_Fake_Method", log_nth=100)

[Iteration 1/32] TEST loss: 0.49971136450767517

[TEST] mean acc/loss: 0.771484375/0.5277904272079468


In [107]:
# Load test data for one specific (different) fake type

fake_type = "Pristine" # NeuralTextures, FaceSwap, Face2Face, Deepfakes, Pristine

# because we constructed two different folders, one with 100 videos, one with 1000 videos and the directory structure is different
if is_large_dataset:
    dif_test_data_location = [root_path + "/original_sequences/youtube/c40/" + sequence + "/test",
                              root_path + "/manipulated_sequences/" + fake_type + "/c40/" + sequence + "/test"
                              ]
else:
    dif_test_data_location = [root_path + "/FaceForensics_Testset/original_sequences/youtube/c40/" + sequence,
                              #root_path + "/FaceForensics_Testset/manipulated_sequences/" + fake_type + "/c40/" + sequence
                              ]

dif_test_dataset = FaceForensicsImagesDataset(dif_test_data_location, transform=ToTensor())

dif_test_indices = range(len(dif_test_dataset))

dif_test_sampler = SubsetRandomSampler(dif_test_indices)

dif_test_loader = torch.utils.data.DataLoader(dif_test_dataset,
                                              batch_size=dataset_args["batch_size"], 
                                              sampler=dif_test_sampler,
                                              num_workers=4)

print("Length of dif fake test set: {}".format(len(dif_test_dataset)))

Length of dif fake test set: 500


In [108]:
solver.test(model, dif_test_loader, test_prefix="Different_Fake_Method_" + fake_type, log_nth=100)

[Iteration 1/16] TEST loss: 0.5336543321609497

[TEST] mean acc/loss: 0.732421875/0.553930938243866


# Save the model

Save network with its weights to disk.

See torch.save function: https://pytorch.org/docs/stable/notes/serialization.html#recommend-saving-models 

Load again with `the_model = TheModelClass(*args, **kwargs) the_model.load_state_dict(torch.load(PATH))`

In [0]:
def save_model(modelname, model):
    filepath = root_dir + "/adl4cv/saved_results/models/" + modelname + ".pt"
    torch.save(model.state_dict(), filepath)

In [0]:
save_model("baseline_" + id_suffix, model)

In [0]:
# LOAD MODEL AGAIN for verification purposes
# Should print: <All keys matched successfully>

#filepath = root_dir + "/adl4cv/saved_results/models/" + "temporal_encoder_2_" + id_suffix + ".pt"
filepath = root_dir + "/adl4cv/saved_results/models/" + "baseline_2020-Jan-22_19-00-31_7624792e-3d49-11ea-a860-0242ac1c0002.pt"
model.load_state_dict(torch.load(filepath))

<All keys matched successfully>